In [61]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns


from DataLoader import DataLoader as DL

import statsmodels.tsa.api as sta
import statsmodels.tsa.stattools as stattools
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import statsmodels.stats.stattools as stools

import scipy.stats as stats
## Harsh code for regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

def norm_rank(kk):
    ## for every row, rank from small to big, then minus 0.5, 
    ## then divide number of data in row
    ## ppf: inverse function of cdf
    ## - 0.5 make sure no 1 or 0 in the ppf
    tmp = stats.norm.ppf((kk.rank(1)-.5).div(kk.count(1), 0))
    ## set index and columns
    kk = pd.DataFrame(tmp, index=kk.index, columns=kk.columns)
    ## normalization
    kk = kk.div(kk.std(1,ddof=0), 0)
    return kk

def get_cross_section(df,col_name):
    tt = df.stack().reset_index().rename(columns={0: col_name})
    return tt

def get_independent_variable(df):
    
    tt = np.log(df.loc[st:,cols].rolling(20).mean())
    tt = norm_rank(tt).stack().reset_index().rename(columns={0: 'col_name'})
    
    return tt


loc='C:/Users/Biao Feng/source/Anaconda3/practicum/'
d = DL(loc)

##load all fundimental variables
for i in range(0,len(d.cols_q)):
    globals()[d.cols_q[i]] = d.get_time_series(d.merged_table, d.cols_q[i], icol='datadate')

mcap = d.get_time_series(d.merged_table, 'mcap')
ret = d.get_time_series(d.merged_table, 'RET')

## if data's mcap is bigger than breakpoint
d.merged_table.loc[:, 'in_universe']=[np.nan if pd.isnull(m) or pd.isnull(b) or m < b else 1 
                                      for m,b in zip(d.merged_table.mcap.values, d.merged_table.breakpt.values)]

# use data only after 1977
st = '1977'

industry = ['i1','i2','i3','i4','i5','i6','i7','i8']
adjusted_score = []
def adj_r2_score(model,y,yhat):
    adj = 1 - float(len(y)-1)/(len(y)-len(model.coef_)-1)*(1 - r2_score(y,yhat))
    return adj
    
#make example by industry 11
#for industry in industry:
industry = 'i1'
#get industry universe
in_universe = d.get_time_series(d.merged_table[d.merged_table[industry]==1], 'in_universe')
in_univ = in_universe[st:].fillna(method='ffill', limit=3)
cols = in_univ.count()[in_univ.count()>20].index

##dependent variable
# book value growth
equity = (atq.loc[st:,cols]-ltq.loc[st:,cols]).rolling(20).mean()
equity_chg = np.log(equity).diff()
y_equchg = norm_rank(equity_chg).stack().reset_index().rename(columns={0: 'book_value growth'})
# book to market
b2m = np.log(equity / mcap * 1000)
y_b2m = norm_rank(b2m).stack().reset_index().rename(columns={0: 'book to market'})

##independent variable
for i in range(0,len(d.cols_q)):
    # 5 year mean
    tt = np.log(globals()[d.cols_q[i]].loc[st:,cols].rolling(20).mean())
    globals()['x_mean_' + d.cols_q[i]] = norm_rank(tt).stack().reset_index().rename(columns={0: d.cols_q[i]+'_mean'})
    # 5 year std
    ff = np.log(globals()[d.cols_q[i]].loc[st:,cols].rolling(20).std())
    globals()['x_std_' + d.cols_q[i]] = norm_rank(ff).stack().reset_index().rename(columns={0: d.cols_q[i]+'_std'})

## get regression penal
## use method 'left' have NaN, cannot regress, so use 'inner', but too less row
reg_penal = pd.merge(y_equchg,y_b2m,on=['gvkey', 'datadate'], how ='inner')
for i in range(0,len(d.cols_q)):
    ##  threshold for independent variable, if data number is less than it, 
    ##  do not merge in the reg_penal
    ## threshold should be changed depend on different industry
    if len(globals()['x_mean_' + d.cols_q[i]]) > 900:
        reg_penal = pd.merge(reg_penal, globals()['x_mean_' + d.cols_q[i]],
                             on=['gvkey', 'datadate'], how ='inner')
        reg_penal = pd.merge(reg_penal, globals()['x_std_' + d.cols_q[i]],
                             on=['gvkey', 'datadate'], how ='inner')

#####################################################################################
## regression from now

X = reg_penal.iloc[:,4:].values
y = reg_penal['book_value growth'].values        

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size=0.2, random_state=42)
lm = LinearRegression()
lm.fit(X_train,y_train)
y_train_pred = lm.predict(X_train)
y_test_pred = lm.predict(X_test)
print('R square score',r2_score(y_test, y_test_pred))
print('adjusted R squared score',adj_r2_score(lm, y_test,y_test_pred))
    #adjusted_score.append(adj_r2_score(lm, y_test,y_test_pred))
#adjusted = pd.DataFrame([adjusted_score], columns = industry)

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Biao Feng\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: RuntimeWarning: invalid value encountered in log
C:\Users\Biao Feng\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
C:\Users\Biao Feng\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1904: RuntimeWarn

R square score 0.9751975775579671
adjusted R squared score 1.0021434192233856


C:\Users\Biao Feng\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
C:\Users\Biao Feng\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in log
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in log
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in log


R square score 0.7994575314552362
adjusted R squared score -1.286184141410307


C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: RuntimeWarning: invalid value encountered in log
C:\Users\Biao Feng\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
C:\Users\Biao Feng\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in log
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in log
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in log
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in log


ValueError: Found array with 0 sample(s) (shape=(0, 94)) while a minimum of 1 is required.

In [58]:
X = reg_penal.iloc[:,4:].values
y = reg_penal['book_value growth'].values        

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size=0.2, random_state=42)
print(X_train)

[]


In [60]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns


from DataLoader import DataLoader as DL

import statsmodels.tsa.api as sta
import statsmodels.tsa.stattools as stattools
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import statsmodels.stats.stattools as stools

import scipy.stats as stats

def norm_rank(kk):
    ## for every row, rank from small to big, then minus 0.5, 
    ## then divide number of data in row
    ## ppf: inverse function of cdf
    ## - 0.5 make sure no 1 or 0 in the ppf
    tmp = stats.norm.ppf((kk.rank(1)-.5).div(kk.count(1), 0))
    ## set index and columns
    kk = pd.DataFrame(tmp, index=kk.index, columns=kk.columns)
    ## normalization
    kk = kk.div(kk.std(1,ddof=0), 0)
    return kk

def get_cross_section(df,col_name):
    tt = df.stack().reset_index().rename(columns={0: col_name})
    return tt

def get_independent_variable(df):
    
    tt = np.log(df.loc[st:,cols].rolling(20).mean())
    tt = norm_rank(tt).stack().reset_index().rename(columns={0: 'col_name'})
    
    return tt


loc='C:/Users/hecha/Downloads/practicum/main_work/'
d = DL(loc)

##load all fundimental variables
for i in range(0,len(d.cols_q)):
    globals()[d.cols_q[i]] = d.get_time_series(d.merged_table, d.cols_q[i], icol='datadate')

mcap = d.get_time_series(d.merged_table, 'mcap')
ret = d.get_time_series(d.merged_table, 'RET')

## if data's mcap is bigger than breakpoint
d.merged_table.loc[:, 'in_universe']=[np.nan if pd.isnull(m) or pd.isnull(b) or m < b else 1 
                                      for m,b in zip(d.merged_table.mcap.values, d.merged_table.breakpt.values)]

# use data only after 1977
st = '1977'

#make example by industry 11
industry = 'i2'

#get industry universe
in_universe = d.get_time_series(d.merged_table[d.merged_table[industry]==1], 'in_universe')
in_univ = in_universe[st:].fillna(method='ffill', limit=3)
cols = in_univ.count()[in_univ.count()>20].index

##dependent variable
# book value growth
equity = (atq.loc[st:,cols]-ltq.loc[st:,cols]).rolling(20).mean()
equity_chg = np.log(equity).diff()
y_equchg = norm_rank(equity_chg).stack().reset_index().rename(columns={0: 'book_value growth'})
# book to market
b2m = np.log(equity / mcap * 1000)
y_b2m = norm_rank(b2m).stack().reset_index().rename(columns={0: 'book to market'})

##independent variable
for i in range(0,len(d.cols_q)):
    # 5 year mean
    tt = np.log(globals()[d.cols_q[i]].loc[st:,cols].rolling(20).mean())
    globals()['x_mean_' + d.cols_q[i]] = norm_rank(tt).stack().reset_index().rename(columns={0: d.cols_q[i]+'_mean'})
    # 5 year std
    ff = np.log(globals()[d.cols_q[i]].loc[st:,cols].rolling(20).std())
    globals()['x_std_' + d.cols_q[i]] = norm_rank(ff).stack().reset_index().rename(columns={0: d.cols_q[i]+'_std'})

## get regression penal
## use method 'left' have NaN, cannot regress, so use 'inner', but too less row
reg_penal = pd.merge(y_equchg,y_b2m,on=['gvkey', 'datadate'], how ='inner')
for i in range(0,len(d.cols_q)):
    ##  threshold for independent variable, if data number is less than it, 
    ##  do not merge in the reg_penal
    ## threshold should be changed depend on different industry
    if len(globals()['x_mean_' + d.cols_q[i]]) > 900:
        reg_penal = pd.merge(reg_penal, globals()['x_mean_' + d.cols_q[i]],
                             on=['gvkey', 'datadate'], how ='inner')
        reg_penal = pd.merge(reg_penal, globals()['x_std_' + d.cols_q[i]],
                             on=['gvkey', 'datadate'], how ='inner')



#####################################################################################
## regression from now

## Harsh code for regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

X = reg_penal.iloc[:,4:].values
y = reg_penal['book_value growth'].values        

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size=0.4, random_state=101)

lm = LinearRegression()
lm.fit(X_train,y_train)

y_train_pred = lm.predict(X_train)
y_test_pred = lm.predict(X_test)

plt.scatter(y_test,y_test_pred)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/hecha/Downloads/practicum/main_work/link table 1962-2001_csv.zip'